## 1. module import 

In [22]:
import os
import sys
import argparse
import numpy as np
import time
import glob
import cv2

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('/home/juju/contact_graspnet/contact_graspnet/inference.py')))
sys.path.append(os.path.join(BASE_DIR))
sys.path.append(os.path.join('/home/juju/contact_graspnet/contact_graspnet'))
sys.path.append(os.path.join('/home/juju/contact_graspnet'))

import config_utils
from data import regularize_pc_point_count, depth2pc, load_available_input_data



from contact_grasp_estimator import GraspEstimator
from visualization_utils import visualize_grasps, show_image

In [23]:
sys.path

['/home/juju/jh_study/contact_graspnet_study',
 '/home/juju/jh_study/contact_graspnet_study',
 '/home/juju/HPE/ViTPose',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python38.zip',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python3.8',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python3.8/lib-dynload',
 '',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python3.8/site-packages',
 '/home/juju/contact_graspnet',
 '/home/juju/contact_graspnet/contact_graspnet',
 '/home/juju/contact_graspnet',
 '/home/juju/contact_graspnet/contact_graspnet',
 '/home/juju/contact_graspnet/pointnet2/tf_ops/grouping',
 '/home/juju/contact_graspnet/pointnet2/utils',
 '/home/juju/contact_graspnet/pointnet2/utils',
 '/home/juju/contact_graspnet',
 '/home/juju/contact_graspnet/pointnet2/utils',
 '/home/juju/contact_graspnet/contact_graspnet/contact_grasp_estimator.py',
 '/home/juju/contact_graspnet',
 '/home/juju/contact_graspnet/contact_graspnet',
 '/home/juju/contact_graspnet',
 '/home/juju/con

## 2. argument setting

In [51]:
parser = argparse.ArgumentParser()
parser.add_argument('--ckpt_dir', default='checkpoints/scene_test_2048_bs3_hor_sigma_001', help='Log dir [default: checkpoints/scene_test_2048_bs3_hor_sigma_001]')
parser.add_argument('--np_path', default='test_data/7.npy', help='Input data: npz/npy file with keys either "depth" & camera matrix "K" or just point cloud "pc" in meters. Optionally, a 2D "segmap"')
parser.add_argument('--png_path', default='', help='Input data: depth map png in meters')
parser.add_argument('--K', default=None, help='Flat Camera Matrix, pass as "[fx, 0, cx, 0, fy, cy, 0, 0 ,1]"')
parser.add_argument('--z_range', default=[0.2,1.8], help='Z value threshold to crop the input point cloud')
parser.add_argument('--local_regions', action='store_true', default=False, help='Crop 3D local regions around given segments.')
parser.add_argument('--filter_grasps', action='store_true', default=False,  help='Filter grasp contacts according to segmap.')
parser.add_argument('--skip_border_objects', action='store_true', default=False,  help='When extracting local_regions, ignore segments at depth map boundary.')
parser.add_argument('--forward_passes', type=int, default=1,  help='Run multiple parallel forward passes to mesh_utils more potential contact points.')
parser.add_argument('--segmap_id', type=int, default=0,  help='Only return grasps of the given object id')
parser.add_argument('--arg_configs', nargs="*", type=str, default=[], help='overwrite config parameters')

FLAGS = parser.parse_args(args=[])

## 3. Config setting

In [52]:
global_config = config_utils.load_config(FLAGS.ckpt_dir, batch_size=FLAGS.forward_passes, arg_configs=FLAGS.arg_configs)

global_config

/home/juju/contact_graspnet/contact_graspnet/config_utils.py:40: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  global_config = yaml.load(f)


{'DATA': {'scene_contacts_path': 'scene_contacts',
  'gripper_width': 0.08,
  'input_normals': False,
  'use_uniform_quaternions': False,
  'view_sphere': {'elevation': [30, 150], 'distance_range': [0.9, 1.3]},
  'train_on_scenes': True,
  'labels': {'to_gpu': False,
   'bin_weights': [0.16652107,
    0.21488856,
    0.37031708,
    0.55618503,
    0.75124664,
    0.93943357,
    1.07824539,
    1.19423112,
    1.55731375,
    3.17161779],
   'filter_z': True,
   'k': 1,
   'max_radius': 0.005,
   'num_neg_contacts': 0,
   'num_pos_contacts': 8000,
   'offset_bins': [0,
    0.00794435329,
    0.0158887021,
    0.0238330509,
    0.0317773996,
    0.0397217484,
    0.0476660972,
    0.055610446,
    0.0635547948,
    0.0714991435,
    0.08],
   'z_val': -0.1},
  'pc_augm': {'occlusion_nclusters': 0,
   'occlusion_dropout_rate': 0.0,
   'sigma': 0.0,
   'clip': 0.005},
  'depth_augm': {'sigma': 0.001, 'clip': 0.005, 'gaussian_kernel': 0},
  'raw_num_points': 20000,
  'ndataset_points': 20

In [13]:
# inference 시 사용할 batch_size이기 때문에 batch를 1로 셋팅한다. 
FLAGS.forward_passes

1

###### config_utils.load_config 코드 분석

In [53]:
import os
import yaml

def recursive_key_value_assign(d,ks,v):
    """
    Recursive value assignment to a nested dict

    Arguments:
        d {dict} -- dict
        ks {list} -- list of hierarchical keys
        v {value} -- value to assign
    """
    
    if len(ks) > 1:
        recursive_key_value_assign(d[ks[0]],ks[1:],v)
    elif len(ks) == 1:
        d[ks[0]] = v
        
def load_config(checkpoint_dir, batch_size=None, max_epoch=None, data_path=None, arg_configs=[], save=False):
    """
    Loads yaml config file and overwrites parameters with function arguments and --arg_config parameters

    Arguments:
        checkpoint_dir {str} -- Checkpoint directory where config file was copied to

    Keyword Arguments:
        batch_size {int} -- [description] (default: {None})
        max_epoch {int} -- "epochs" (number of scenes) to train (default: {None})
        data_path {str} -- path to scenes with contact grasp data (default: {None})
        arg_configs {list} -- Overwrite config parameters by hierarchical command line arguments (default: {[]})
        save {bool} -- Save overwritten config file (default: {False})

    Returns:
        [dict] -- Config
    """

    config_path = os.path.join(checkpoint_dir, 'config.yaml')
#     print(config_path)
#     config_path = config_path if os.path.exists(config_path) else os.path.join(os.path.dirname(__file__),'config.yaml')
    config_path = '/home/juju/contact_graspnet/checkpoints/scene_test_2048_bs3_hor_sigma_001/config.yaml'
    with open(config_path,'r') as f:
        global_config = yaml.load(f)

    for conf in arg_configs:
        k_str, v = conf.split(':')
        try:
            v = eval(v)
        except:
            pass
        ks = [int(k) if k.isdigit() else k for k in k_str.split('.')]
        
        recursive_key_value_assign(global_config, ks, v)
        
    if batch_size is not None:
        global_config['OPTIMIZER']['batch_size'] = int(batch_size)
    if max_epoch is not None:
        global_config['OPTIMIZER']['max_epoch'] = int(max_epoch)
    if data_path is not None:
        global_config['DATA']['data_path'] = data_path
        
    global_config['DATA']['classes'] = None
    
    if save:
        with open(os.path.join(checkpoint_dir, 'config.yaml'),'w') as f:
            yaml.dump(global_config, f)

    return global_config

In [54]:
global_config = load_config(FLAGS.ckpt_dir, batch_size=FLAGS.forward_passes, arg_configs=FLAGS.arg_configs)


/tmp/ipykernel_45778/2857623346.py:42: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  global_config = yaml.load(f)


In [56]:
checkpoint_dir = '/home/juju/contact_graspnet/checkpoints/scene_test_2048_bs3_hor_sigma_001/config.yaml'
config_path = os.path.join(checkpoint_dir, 'config.yaml')

os.path.exists(checkpoint_dir)

True

In [59]:
global_config['OPTIMIZER']['batch_size']

1